# Setup

Run these blocks each time the file is opened or runtime is restarted

Runtime must be restarted each time a different model is run

In [ ]:
#import packages
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
#mount google drive
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#check gpu specifications
!nvidia-smi

Mon Apr 24 18:45:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#download model
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 186Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.63Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 252Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:13, 37.9Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 151Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.77Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.32Mit/s]


# WOW Quest Generator

In [ ]:
#load WOW quest dataset
!wget jakub.thebias.nl/GPT2_WOWHead_dataset.txt

--2023-04-03 15:18:38--  http://jakub.thebias.nl/GPT2_WOWHead_dataset.txt
Resolving jakub.thebias.nl (jakub.thebias.nl)... 130.89.161.150, 2001:67c:2564:331:6ab5:99ff:fe6d:772c
Connecting to jakub.thebias.nl (jakub.thebias.nl)|130.89.161.150|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jakub.thebias.nl/GPT2_WOWHead_dataset.txt [following]
--2023-04-03 15:18:38--  https://jakub.thebias.nl/GPT2_WOWHead_dataset.txt
Connecting to jakub.thebias.nl (jakub.thebias.nl)|130.89.161.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11704501 (11M) [text/plain]
Saving to: ‘GPT2_WOWHead_dataset.txt’

GPT2_WOWHead_datase 100%[===================>]  11.16M  9.06MB/s    in 1.2s    

2023-04-03 15:18:40 (9.06 MB/s) - ‘GPT2_WOWHead_dataset.txt’ saved [11704501/11704501]



###Fine-Tuning

In [ ]:
#fine-tuning takes about 30min using 128M model on T4GPU
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset="GPT2_WOWHead_dataset.txt",
              model_name='124M',
              steps=1000, #number of steps to run for
              restore_from='fresh', #fresh means to train from base gpt, use latest to start from an existing checpoint
              run_name='run1', #subfolder within checkpoint to save the model
              print_every=10,#Number of steps to print training progress
              sample_every=200,#Number of steps to print example output
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:12<00:00, 12.63s/it]


dataset has 3119753 tokens
Training...
[10 | 29.79] loss=2.71 avg=2.71
[20 | 51.72] loss=2.72 avg=2.71
[30 | 74.04] loss=2.23 avg=2.55
[40 | 96.78] loss=2.81 avg=2.62
[50 | 119.92] loss=2.62 avg=2.62
[60 | 143.50] loss=2.63 avg=2.62
[70 | 167.31] loss=2.55 avg=2.61
[80 | 190.79] loss=2.59 avg=2.61
[90 | 214.27] loss=2.65 avg=2.61
[100 | 237.93] loss=2.56 avg=2.61
[110 | 261.53] loss=2.46 avg=2.59
[120 | 285.08] loss=2.61 avg=2.59
[130 | 308.63] loss=2.70 avg=2.60
[140 | 332.26] loss=2.69 avg=2.61
[150 | 355.90] loss=2.28 avg=2.59
[160 | 379.51] loss=2.42 avg=2.57
[170 | 403.08] loss=2.07 avg=2.54
[180 | 426.68] loss=2.28 avg=2.53
[190 | 450.27] loss=2.44 avg=2.52
[200 | 473.81] loss=2.47 avg=2.52
======== SAMPLE 1 ========
 represent the first three, as we make our way through this valley. As you see, there is the first three trees to the east, along with three additional trees. Return to the hut.<|endoftext|>
<|startoftext|>Jared's Last Stand<|obj|>Slay 5 of Jared's Jellins in the Rui

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
#save dataset checkpoint to google drive
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Once Model Has been created, above lines dont need to be run again for testing

In [ ]:
# load the model from google drive
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000


### Generate Text From The Fine-Tuned Model

In [ ]:
gpt2.generate(sess, run_name='run1')

You've been to the Ruins of Kraul, a place where all manner of evil runs rampant. We believe the Twilight's Hammer has uncovered a hidden location of great power. You must travel there. Bring me back a sample of the demonic blood you seek!<|endoftext|>
<|startoftext|>The Shadow Vault<|obj|>Kill 6 Twilight's Hammer Cultists in the Shadow Vault.<|text|>The Twilight's Hammer cultists have recently discovered a new location of great power to be found out there. Your mission is a simple one - infiltrate this location and destroy as many cultists as you can.<|endoftext|>
<|startoftext|>The Twilight's Hammer<|obj|>Kill 6 Twilight's Hammer Cultists in the Shadow Vault.<|text|>The Twilight's Hammer cultists have recently discovered a new location of great power to be found out there. Your mission is a simple one - infiltrate this location and destroy as many cultists as you can.<|endoftext|>
<|startoftext|>The Shadow Vault<|obj|>Kill 6 Twilight's Hammer Cultists in the Shadow Vault.<|text|>The 

In [ ]:
#generated text results for specifc quest name and objective
# format <|startoftext|> Quest Name <|obj|> Quest Objective <|text|>
gpt2.generate(sess,
              length=250, #number of tokens to generate
              temperature=0.7,#level of text unpredictability
              prefix="<|startoftext|>Spooky Cave Party<|obj|>Talk to the goblins in the Big Spooky Cave.<|text|>", #prefix to generate text off of
              truncate="<|endoftext|>", #truncate input text when encountering this sequence
              nsamples=5, #number of output text samples to show
              batch_size=5,
              include_prefix=False, #hides prefix in output sample text
              )

Hey, you're not here for a ride, are you? We're all out here trying to find the treasure chest that's hidden deep in this cave. I'm looking for a goblin to play with and help me with an overstuffing cave party. If you can go talk to the goblins in the cave, I'm sure they'll be warm to see you.
I've got a little talk with the goblins in the cave. I hope you don't mind them.
We're going to need a lot of help to survive in the wild!
The goblins have been looking for a place to live, and for something to do with Goblins.
The goblins are out there, and they're looking for something to do with goblins.
Talk to one of the goblins and see if you can get us some help from the locals.

I don't like goblins.
These gnolls are really bad and they're making us jump out of bed like they're going in!
I need the rest of our men to help me out!
They need to be moved to the Big Spooky Cave to the north. We'll keep an eye out for them!
Lucky for us, the goblins have set up a no-good troll party for a few 

In [ ]:
#try it again using the specific quest description name, objective and parameters provided in the original paper's code for comparison
gpt2.generate(sess,
              run_name='run1',
              temperature=0.9,
              prefix="<|startoftext|>In Dire Need<|obj|>Hear out the Council of Six in the Purple Parlor.<|text|>",
              truncate="<|endoftext|>",
              nsamples=5,
              batch_size=5,
              include_prefix=False,
              )

One of the Council of Six is preparing an envoy and demanding that we communicate with her.
Sooner or later, the Council will dismiss her from her council.
While she is of service, you should be the one to speak to her and make sure that conversation goes smoothly.
If anything came out of the Council that happened, the problem will be one of vermin. Vermin. Undesignated vermin. When the torch goes out, the damned purple have come back in. Will you listen to me?
It is time when the Council of Six met with Captain Taylor in the Purloining Hall.
I believe he'll be a nice surprise to see again.
The Council of Six is the last of new members of the new council, given the conflict between us. Bringing out a new member does not change the little goblin.
Our boys have discovered a dire need from afar. Our scouts report the talkers have made their way to the Temple of Drak'Thalas in southern Durotar and informed them of a spire that can accommodate four times that number of men. Unfortunately, t

# Running on the SWTOR Database

###Fine-Tuning the Checkpoint on the SWTOR Database

In [ ]:
#make new session, first try fine-tuning on current checkpoint
#note that the previous WOW checkpoint must have been created for this to work
#make sure to donwload model again at top if not downlaoded
#fine-tuning takes about 30min using 128M model on T4GPU
sess2 = gpt2.start_tf_sess()

gpt2.finetune(sess2,
              dataset="/content/drive/MyDrive/swtor_final_dataset.txt",
              model_name='124M',
              steps=1000, #number of steps to run for
              restore_from='latest', #fresh means to train from base gpt, use latest to start from an existing checpoint
              run_name='run2', #subfolder within checkpoint to save the model
              print_every=10,#Number of steps to print training progress
              sample_every=200,#Number of steps to print example output
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


dataset has 242615 tokens
Training...
[10 | 26.81] loss=2.45 avg=2.45
[20 | 48.84] loss=2.29 avg=2.37
[30 | 71.37] loss=2.17 avg=2.30
[40 | 93.80] loss=2.30 avg=2.30
[50 | 116.00] loss=2.05 avg=2.25
[60 | 138.21] loss=1.90 avg=2.19
[70 | 160.55] loss=1.99 avg=2.16
[80 | 182.88] loss=1.87 avg=2.12
[90 | 205.18] loss=1.57 avg=2.06
[100 | 227.44] loss=1.69 avg=2.02
[110 | 249.67] loss=1.58 avg=1.98
[120 | 271.93] loss=1.59 avg=1.94
[130 | 294.21] loss=1.61 avg=1.92
[140 | 316.51] loss=1.27 avg=1.87
[150 | 338.82] loss=1.35 avg=1.83
[160 | 361.11] loss=1.54 avg=1.81
[170 | 383.41] loss=1.38 avg=1.78
[180 | 405.68] loss=1.50 avg=1.77
[190 | 427.94] loss=1.52 avg=1.75
[200 | 450.19] loss=1.21 avg=1.72
======== SAMPLE 1 ========
 of the Star of Coruscant. Use your ship’s galaxy map to travel to the planet of Voss.<|endoftext|>
<|startoftext|>Final Stage – Ending a Better Life<|obj|>Defeat Separatist Forces<|text|>The separatists in Voss are making very clear they want to take the Republic awa

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
#save dataset checkpoint to google drive
gpt2.copy_checkpoint_to_gdrive(run_name='run2')

Once Model Has been created,above lines dont need to be run again for testing

In [ ]:
# load the model from google drive
gpt2.copy_checkpoint_from_gdrive(run_name='run2')

sess2 = gpt2.start_tf_sess()
gpt2.load_gpt2(sess2, run_name='run2')

Loading checkpoint checkpoint/run2/model-1000


In [ ]:
gpt2.generate(sess2, run_name='run2')

>The Shadow Syndicate<|obj|>Find the Second Hostile Situation<|text|>The ICC has become involved in the government of General Aves. Suspicion of General Aves’ whereabouts has been tested at this point, but resistance has grown in numbers as the conflict between the Imperials and the Shadow Syndicate grows. General Aves has asked you to find a defector from the General and recover the ICC datapad. A bodyguard is provided as proof of his demise. Retrieve the ICC datapad from the body of a defector.<|endoftext|>
<|startoftext|>The Master’s Secret<|obj|>Use the Holoterminal<|text|>At last our targets have been defeated, and the final confrontation between Darth Baras and the Jedi Masters on Taris has begun. The Jedi have traced a crashed dreadnaught to the Voss, and the Empire’s advance is dangerously close. The Master of the Voss, Tulak Hord, has asked you to rescue the master’s lightsaber. Rance the White Seraph, and obtain the master’s holocall. The White Seraph must be in the area. Ent

In [ ]:
gpt2.generate(sess2,
              run_name='run2',
              length=250, #number of tokens to generate
              temperature=0.7,#level of text unpredictability
              prefix="<|startoftext|>The Frozen War<|obj|>Talk to the wampas on Hoth.<|text|>", #prefix to generate text off of
              truncate="<|endoftext|>", #truncate input text when encountering this sequence
              nsamples=5, #number of output text samples to show
              batch_size=5,
              include_prefix=False, #hides prefix in output sample text
              )

Your mission on Hoth is to recruit a technical specialist named Yuun into Havoc Squad. Yuun is currently assigned to recreate an Umbra encrypter, the device responsible for decades of unbreakable Imperial codes, using parts from crashed warships spread across Hoth’s frozen wastes. Sergeant Yuun has determined that the final Umbra component has been taken to the Star of Coruscant, a fallen dreadnaught occupied by the White Maw pirates. Unfortunately, Sergeant Yuun’s squad has suffered heavy casualties, and they are unable to assist you in recovering the component. Travel to the Star of Coruscant in the Starship Graveyard and recover the final Umbra component.
The Republic is fighting to win favor with the Voss, the inhabitants of a newly discovered and mysterious world. You have been asked to assist Ambassador Jannik in his efforts to negotiate an alliance. After exposing a Sith plot to recruit failed Mystics, the Voss envoy Laryn-Ki revealed that the Mystics have had a vision. You will

Fine-Tuning the Base Model on the SWTOR Database

In [ ]:
#make new session, fine-tuning on base model
#previous checkpoints don't need to have been created for this to work
#make sure to donwload model again at top if not downlaoded
#fine-tuning takes about 30min using 128M model on T4GPU
sess3 = gpt2.start_tf_sess()

gpt2.finetune(sess3,
              dataset="/content/drive/MyDrive/swtor_final_dataset.txt",
              model_name='124M',
              steps=1000, #number of steps to run for
              restore_from='fresh', #fresh means to train from base gpt, use latest to start from an existing checpoint
              run_name='run3', #subfolder within checkpoint to save the model
              print_every=10,#Number of steps to print training progress
              sample_every=200,#Number of steps to print example output
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


dataset has 242615 tokens
Training...
[10 | 29.77] loss=2.38 avg=2.38
[20 | 51.23] loss=2.23 avg=2.30
[30 | 72.95] loss=2.04 avg=2.22
[40 | 95.09] loss=2.25 avg=2.22
[50 | 117.08] loss=1.94 avg=2.17
[60 | 138.93] loss=1.95 avg=2.13
[70 | 160.86] loss=1.81 avg=2.08
[80 | 182.82] loss=1.79 avg=2.05
[90 | 204.79] loss=1.77 avg=2.01
[100 | 226.80] loss=1.56 avg=1.97
[110 | 248.73] loss=1.79 avg=1.95
[120 | 270.64] loss=1.77 avg=1.93
[130 | 292.51] loss=1.54 avg=1.90
[140 | 314.36] loss=1.46 avg=1.87
[150 | 336.36] loss=1.44 avg=1.84
[160 | 358.24] loss=1.53 avg=1.82
[170 | 380.16] loss=1.41 avg=1.79
[180 | 402.03] loss=1.43 avg=1.77
[190 | 424.01] loss=1.34 avg=1.74
[200 | 445.95] loss=1.12 avg=1.71
======== SAMPLE 1 ========
While the Republic is trying to defeat Corellia, you need to stop them before they destroy everything you stand in your way. Defeat the Corellian forces inside the Corellian prison on Ortolani.<|endoftext|>
<|startoftext|>Clear Out the Prison Facility<|obj|>Defeat Rep

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
#save dataset checkpoint to google drive
gpt2.copy_checkpoint_to_gdrive(run_name='run3')

Once Model Has been created,above lines dont need to be run again for testing

In [ ]:
# load the model from google drive
gpt2.copy_checkpoint_from_gdrive(run_name='run3')

sess3 = gpt2.start_tf_sess()
gpt2.load_gpt2(sess3, run_name='run3')

Loading checkpoint checkpoint/run3/model-1000


In [ ]:
gpt2.generate(sess3, run_name='run3')

Seek Your Contact<|obj|>Travel to Nar Shaddaa<|text|>You have been contacted by a Data Corp representative who has information that can help you defeat Darth Jadus. The representative, in order to gain access to the data center, has asked you to steal a data spike the representative is using to trap Darth Jadus. Destroy the data spike in the Data Corp outside the Data Corp Mobilization Center.<|endoftext|>
<|startoftext|>Crystal Ball<|obj|>Infiltrate the First Meeting<|text|>A representative from the First Disciple, a powerful warrior cult that stands between the forces of Darth Jadus and the Jedi of old, has decided to sneak a meeting with the leaders of the first to come. The representative, in a bid to hide his identity, has enlisted your help. Place the acoustic disruptors around the arena to drown out the First Disciple.<|endoftext|>
<|startoftext|>First to Disciples<|obj|>Infiltrate the Temple of Corruption<|text|>The meeting with the Jedi will continue in the Fallenspire Strongh

In [ ]:
gpt2.generate(sess3,
              run_name='run3',
              length=250, #number of tokens to generate
              temperature=0.7,#level of text unpredictability
              prefix="<|startoftext|>The Frozen War<|obj|>Talk to the wampas on Hoth.<|text|>", #prefix to generate text off of
              truncate="<|endoftext|>", #truncate input text when encountering this sequence
              nsamples=5, #number of output text samples to show
              batch_size=5,
              include_prefix=False, #hides prefix in output sample text
              )

You have been tasked with recovering data spikes from an SIS agent lost to a local Killik hive. As you make your way to the hive, you’ll need to thin the Killiks’ numbers. Use the hive’s heat exchanger to heat the Killiks inside the Ruur Killik Burrows.
You have been tasked with retrieving technological advances from the Sith and turning the warlord into a machine-warlord. As you search for the technology, Peblak Or has informed you that the White Maw pirates are using advanced mining equipment to mine the rakghouls.
You have joined Darth Gravus’s campaign to crush the Republic’s reconstruction of Taris. After defeating their guerrilla commander and ending Republic attempts to control the rakghouls, Darth Gravus needs you to break a stalemate against an army of entrenched Imperial soldiers and a resistance force of Elonian settlers. To aid the assault against the Imperial forces, collect thermite cryo containers from wampas in the Western Clabburn Tundra and freeze any Imperial soldier